# Stock purchase recommendations with Machine Learning - Model Training

In [1]:
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook # progress bar
import fastparquet
import pickle

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

from dask.distributed import Client, progress
from sklearn.externals import joblib

In [25]:
pd.set_option('display.max_columns', 1500)

## 1) Load Training and Test Data

In [2]:
# load the training and test datae from feature engineering step:
X_train = fastparquet.ParquetFile('../data/processed/X_train.parq').to_pandas()
X_test = fastparquet.ParquetFile('../data/processed/X_test.parq').to_pandas()
y_train = pickle.load(open('../data/processed/y_train.pkl', 'rb'))
y_test = pickle.load(open('../data/processed/y_test.pkl', 'rb'))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7024, 693), (1756, 693), (7024,), (1756,))

In [3]:
# quick inspection
X_train.tail()

,AdjVolume_-19,AdjVolume_-18,AdjVolume_-17,AdjVolume_-16,AdjVolume_-15,AdjVolume_-14,AdjVolume_-13,AdjVolume_-12,AdjVolume_-11,AdjVolume_-10,...,weekday,day,AAPL.US,ABBV.US,AMZN.US,CSCO.US,GE.US,INTC.US,MSFT.US,NFLX.US
Index,,,,,,,,,,,,,,,,,,,,,
2017-05-10,1.193791,0.759857,0.816960,0.927417,0.791144,1.113186,0.810204,1.388600,0.931141,0.965937,...,2,10,0,1,0,0,0,0,0,0
2017-05-10,1.479636,0.943690,0.695273,0.775977,0.951766,0.693312,0.806673,0.893978,0.861651,0.904349,...,2,10,0,0,0,1,0,0,0,0
2017-05-10,0.792740,0.694295,0.645960,0.572547,0.675032,0.908420,0.674742,0.666782,0.709628,0.764080,...,2,10,1,0,0,0,0,0,0,0
2017-05-10,1.178886,1.523721,1.370172,1.439560,1.377645,1.350862,1.161777,1.496622,1.616429,1.229982,...,2,10,0,0,1,0,0,0,0,0
2017-05-10,0.958888,1.003052,0.935390,0.849433,1.512875,1.249828,1.822810,1.665799,1.680479,1.431698,...,2,10,0,0,0,0,0,0,1,0


In [4]:
X_test.head()

,AdjVolume_-19,AdjVolume_-18,AdjVolume_-17,AdjVolume_-16,AdjVolume_-15,AdjVolume_-14,AdjVolume_-13,AdjVolume_-12,AdjVolume_-11,AdjVolume_-10,...,weekday,day,AAPL.US,ABBV.US,AMZN.US,CSCO.US,GE.US,INTC.US,MSFT.US,NFLX.US
Index,,,,,,,,,,,,,,,,,,,,,
2017-05-11,0.639493,0.596356,0.541554,0.964530,0.796825,1.162128,1.062026,1.071385,0.912776,1.151286,...,3,11,0,0,0,0,0,0,1,0
2017-05-11,0.741068,0.590482,0.579214,0.780116,1.079741,0.958561,1.222818,1.029151,1.260323,1.712990,...,3,11,0,0,0,0,0,1,0,0
2017-05-11,0.665995,2.956990,3.554402,2.239698,1.309693,1.233189,0.866047,3.718421,1.661812,1.381076,...,3,11,0,0,0,0,0,0,0,1
2017-05-11,0.991471,0.730477,0.815267,0.999956,0.728416,0.847517,0.939242,0.905279,0.950139,1.596598,...,3,11,0,0,0,1,0,0,0,0
2017-05-11,0.696296,0.647822,0.574197,0.676977,0.911038,0.676686,0.668704,0.711673,0.766282,0.544953,...,3,11,1,0,0,0,0,0,0,0


In [5]:
y_train

Index
2013-11-13    False
2013-11-13    False
2013-11-13    False
2013-11-13     True
2013-11-13     True
2013-11-13    False
2013-11-13     True
2013-11-13    False
2013-11-14    False
2013-11-14    False
2013-11-14     True
2013-11-14    False
2013-11-14     True
2013-11-14    False
2013-11-14     True
2013-11-14    False
2013-11-15     True
2013-11-15    False
2013-11-15    False
2013-11-15    False
2013-11-15     True
2013-11-15     True
2013-11-15     True
2013-11-15    False
2013-11-18    False
2013-11-18    False
2013-11-18     True
2013-11-18     True
2013-11-18     True
2013-11-18    False
              ...  
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-09    False
2017-05-09    False
2017-05-09    False
2017-05-09    False
2017-05-09    

## 2) Build Initial Model - RandomForestClassifier

Start the ML process with a simple out of the box RandomForestClassifier to get a base line and validate that the training data is functioning with sklearn and can generate some predictions.

In [6]:
# define a simple pipeline
pipeline = Pipeline([
    ('standardScaler', StandardScaler()),
    ('randomForest', RandomForestClassifier())
])

# inspect parameters
pipeline.get_params()

{'memory': None,
 'randomForest': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'randomForest__bootstrap': True,
 'randomForest__class_weight': None,
 'randomForest__criterion': 'gini',
 'randomForest__max_depth': None,
 'randomForest__max_features': 'auto',
 'randomForest__max_leaf_nodes': None,
 'randomForest__min_impurity_decrease': 0.0,
 'randomForest__min_impurity_split': None,
 'randomForest__min_samples_leaf': 1,
 'randomForest__min_samples_split': 2,
 'randomForest__min_weight_fraction_leaf': 0.0,
 'randomForest__n_estimators': 'warn',
 'randomForest__n_jobs': None,
 'randomForest__oob_score': F

In [7]:
%%timeit -r1 -n1

# fit the pipeline with all default parameters
pipeline.fit(X_train, y_train.reset_index().setup_for_profitable_trade)

C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


1 loop, best of 1: 4.18 s per loop


In [8]:
# create prediction
y_pred_firstRF = pipeline.predict(X_test)

# save for backtesting in separate notebook
pickle.dump(y_pred_firstRF, open('../data/model_predictions/y_pred_firstRF.pkl', 'wb'))

C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


We now have a first prediction on the dataset - let's look into the performance of the default settings. To understand the result, we will look at a few different metrics: the accuracy, precision, recall, f1-score, and the confusion matrix.

It is important to note that our data is imbalanced (not the same number of days with trade signal as with no signal). Therefore, F scores are a better metric than ROC-AUC or accuracy. We can not easily fix the imbalance since we are looking at time series data and resampling might introduce lookahead bias.

The ultimate test of the quality of the prediction is to backtest the results (ie simulate financial performance based on the predictions). This will be done in the next workbook. For now let's inspect some of the basic metrics.

In [9]:
print(classification_report(y_test, y_pred_firstRF))

              precision    recall  f1-score   support

       False       0.65      0.82      0.72      1153
        True       0.30      0.15      0.20       603

   micro avg       0.59      0.59      0.59      1756
   macro avg       0.47      0.48      0.46      1756
weighted avg       0.53      0.59      0.54      1756



In [10]:
accuracy_score(y_test, y_pred_firstRF)

0.5876993166287016

In [11]:
confusion_matrix(y_test, y_pred_firstRF, labels=[False, True])

array([[940, 213],
       [511,  92]], dtype=int64)

The confusion matrix indicates a large number of False Positives (bad because if we used those to trade, we would enter a trade that turns out not to be as profitable as desired) and False Negatives (bad because they mean missed opportunities for us to enter profitable trades).

The good news is that our out-of-the-box model was able to predict some True Positives but the results in terms of financial performance are most likely very bad (we saved off the prediction and will look at this later).

Even though the model seems to perform poorly, we might be able to learn something from it:

Let's look into the relative feature importances to see if anything stands out

In [12]:
def print_feature_importances(estimator):

    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_train.shape[1]):
        print(str(f + 1) + " importance: " + str(importances[indices[f]]) + ". feature name: " + X_train.columns[indices[f]])

In [13]:
      print_feature_importances(pipeline.named_steps['randomForest'])

Feature ranking:
1 importance: 0.007025686037872939. feature name: AdjCloseSMA50_chg_-15
2 importance: 0.006577991996447342. feature name: AdjCloseSMA50_chg_-19
3 importance: 0.005753848107802519. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-9
4 importance: 0.005269345732750356. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-17
5 importance: 0.005199357066186223. feature name: week
6 importance: 0.005163634957390718. feature name: AdjCloseSMA10_chg_-17
7 importance: 0.005147868505539323. feature name: AdjCloseSMA50_-18
8 importance: 0.005052166896287304. feature name: AdjCloseSMA10_to_AdjClose_ratio_-18
9 importance: 0.005027238092619042. feature name: AdjCloseSMA200_to_AdjClose_ratio_-3
10 importance: 0.004954614786323723. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-12
11 importance: 0.004917220684070975. feature name: AdjCloseSMA200_chg_-5
12 importance: 0.0048579335157953275. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-13
13 importance: 0.004686121717

108 importance: 0.002691557255196896. feature name: AdjCloseSMA10_chg_-19
109 importance: 0.0026878030781793375. feature name: AdjCloseSMA200_-15
110 importance: 0.0026851573295858223. feature name: AdjCloseSMA10_to_AdjClose_ratio_-16
111 importance: 0.0026776773762158495. feature name: AdjClose_chg_-11
112 importance: 0.002671645399426308. feature name: AdjCloseSMA200_chg_-17
113 importance: 0.0026626454738521497. feature name: AdjCloseSMA200_to_AdjClose_ratio_-18
114 importance: 0.002661913747834122. feature name: AdjHigh_chg_-18
115 importance: 0.002646886352015846. feature name: AdjOpen_-18
116 importance: 0.0026360526540459605. feature name: AdjCloseSMA50_-3
117 importance: 0.0026270055427867747. feature name: AdjClose_chg_-2
118 importance: 0.002622691385670515. feature name: AdjClose_chg_-15
119 importance: 0.0026211530354929803. feature name: AdjOpen_chg_-2
120 importance: 0.002609892799118841. feature name: AdjCloseSMA50_-16
121 importance: 0.0026055304615835977. feature name:

219 importance: 0.002027948822979099. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-3
220 importance: 0.0020271568627553654. feature name: AdjCloseSMA200_-7
221 importance: 0.0020081800548742215. feature name: AdjLow_-11
222 importance: 0.0020073663304148626. feature name: AdjCloseSMA50_chg_-12
223 importance: 0.002002702993450044. feature name: AdjLow_chg_-5
224 importance: 0.001996431073785057. feature name: AdjVolume_chg_-2
225 importance: 0.001992314334014823. feature name: AdjLow_chg_-10
226 importance: 0.001974899160777526. feature name: AdjVolume_-12
227 importance: 0.0019743900819357346. feature name: AdjCloseSMA200_chg_-2
228 importance: 0.001971766427527393. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-4
229 importance: 0.0019683957002426707. feature name: AdjCloseSMA50_to_AdjClose_ratio_-3
230 importance: 0.0019680675327455035. feature name: AdjCloseSMA10_-7
231 importance: 0.0019630260934045173. feature name: AdjOpen_-2
232 importance: 0.0019589961836616494. fe

331 importance: 0.0015088803450919231. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-7
332 importance: 0.001506052831468182. feature name: AdjLow_chg_-8
333 importance: 0.0014949612522063627. feature name: GE.US
334 importance: 0.001492996490964338. feature name: AdjCloseSMA200_to_AdjClose_ratio_-5
335 importance: 0.0014843766774278642. feature name: AdjClose_chg_-17
336 importance: 0.001482697341395128. feature name: AdjVolume_chg_-18
337 importance: 0.001481722863360499. feature name: AdjLow_-16
338 importance: 0.0014722466764287296. feature name: AdjCloseSMA10_chg_-8
339 importance: 0.0014721965592142422. feature name: AdjCloseSMA200_to_AdjClose_ratio_-1
340 importance: 0.0014718383247866447. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-18
341 importance: 0.0014636521384759648. feature name: AdjClose_chg_-6
342 importance: 0.001463468340411008. feature name: AdjClose_chg_-9
343 importance: 0.0014623939056301477. feature name: AdjCloseSMA10_chg_-1
344 importance: 0.0014

446 importance: 0.00038525228782011094. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-15
447 importance: 0.00038336447469500485. feature name: AdjClose_crossed_AdjCloseSMA50_-19
448 importance: 0.00037274909481585224. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-19
449 importance: 0.00035083078759450047. feature name: AdjCloseSMA200_to_AdjClose_ratio_-14
450 importance: 0.00033617202898912694. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-13
451 importance: 0.00032561816663757685. feature name: AdjClose_crossed_AdjCloseSMA200_-4
452 importance: 0.0003214599899823406. feature name: AdjClose_crossed_AdjCloseSMA50_-16
453 importance: 0.00031959488515352424. feature name: AdjCloseSMA50_crossed_AdjCloseSMA10_-7
454 importance: 0.0003107838591141425. feature name: AdjClose_above_AdjCloseSMA200_-3
455 importance: 0.00030551405535612084. feature name: AdjClose_crossed_AdjCloseSMA10_-10
456 importance: 0.0002932590086158209. feature name: AdjClose_above_AdjCloseSMA50_-8
457 importa

539 importance: 6.054704957513084e-05. feature name: AdjClose_crossed_AdjCloseSMA200_-6
540 importance: 6.053786543301741e-05. feature name: AdjCloseSMA50_crossed_AdjCloseSMA10_-14
541 importance: 6.0166802403886356e-05. feature name: AdjClose_above_AdjCloseSMA50_-10
542 importance: 5.884498187405539e-05. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-13
543 importance: 5.8714644154482366e-05. feature name: AdjCloseSMA50_crossed_AdjCloseSMA10_-17
544 importance: 5.758281178556969e-05. feature name: AdjCloseSMA50_crossed_AdjCloseSMA10_-4
545 importance: 5.756878029870067e-05. feature name: AdjClose_crossed_AdjCloseSMA200_-18
546 importance: 5.685732333945034e-05. feature name: AdjCloseSMA50_crossed_AdjCloseSMA10_-2
547 importance: 5.674337537855342e-05. feature name: AdjClose_above_AdjCloseSMA50_-11
548 importance: 5.322595693672265e-05. feature name: AdjClose_crossed_AdjCloseSMA200_-1
549 importance: 5.298673191186089e-05. feature name: AdjClose_above_AdjCloseSMA50_-7
550 importan

655 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-12
656 importance: 0.0. feature name: RSI_below_20_-19
657 importance: 0.0. feature name: RSI_above_80_-1
658 importance: 0.0. feature name: RSI_above_80_-3
659 importance: 0.0. feature name: RSI_above_80_-4
660 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA50_-8
661 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA50_-7
662 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA50_-6
663 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA50_-5
664 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA10_-18
665 importance: 0.0. feature name: AdjClose_0
666 importance: 0.0. feature name: AdjCloseSMA50_crossed_AdjCloseSMA10_-16
667 importance: 0.0. feature name: AdjCloseSMA50_crossed_AdjCloseSMA10_-19
668 importance: 0.0. feature name: AdjCloseSMA200_0
669 importance: 0.0. feature name: AdjCloseSMA50_above_AdjCloseSMA10_0
670 importance: 0.0. feature name: AdjCloseSMA50_above_AdjCloseS

At the very bottom of the list are the RSI_above_80 and RSI_below_20 indicators. RSI is an indicator of extremes so having these features not play an important role means that the model did not notice many extremes and having predictive power.
The top section contains many of the SMA50 to SMA200 ratios. The SMA 50 and 200 crossovers or distances are classic momentum indicators (SMA 50 over SMA 200 = uptrend, below = downtrend).
Since the time period in the training data is that of a general market uptrend, it sounds intuitively reasonable that momentum indicators are important.

## 3) Improve RandomForest model with GridSearch

Given the imbalance of our data and the desire to avoid False Positives, we will use a modified F1 score that places stronger emphasis on precision than on recall (they have the same weight in the F1 score). We will skew it towards precision by setting beta to 0.5 and towards recall bysetting beta to 2.0. The combinations were used in separate runs of GridSearchCV and results were saved.

In [14]:
# place higher focus on precision (ie getting most TP and minimize FP) than on recall (ie minimize FN)
# since placing a trade that was based on a FP will be costly and hurts more than missing a trade due to a FN
fhalf_scorer = make_scorer(fbeta_score, beta=0.5) # low beta favors precision over recall
ftwo_scorer = make_scorer(fbeta_score, beta=2) # high beta favors recall over precision

In [15]:
from dask.distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://127.0.0.1:50283 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.46 GB


For a GridSearch, we will define a wide range of parameters to be used in the pipeline. The new fhalf_scorer will be used for optimization.

In [25]:
'''
parameters = {
    'randomForest__min_samples_leaf': [2, 5, 10],
    'randomForest__n_estimators' : [10, 20, 50, 100],
    'randomForest__max_features': [5, 'sqrt', 'log2'], # log2 of 690 = 9, sqrt of 690 = 20
    'randomForest__max_depth' : [4, 5, 6, 7, 8],
    'randomForest__criterion' :['gini', 'entropy']
}
'''
parameters = {
    'randomForest__min_samples_leaf': [2, 5, 10],
    'randomForest__n_estimators' : [10, 20, 50, 100]
    }

my_cv = TimeSeriesSplit(n_splits=3)
cv = GridSearchCV(pipeline, param_grid=parameters, cv=my_cv, scoring='f1', n_jobs=-1, verbose=10) # uses fhalf_scorer

with joblib.parallel_backend('dask'):
    cv.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 4 concurrent workers.
distributed.scheduler - ERROR - 'ndarray-298394c3b8e42a8df373f5caf974b390'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    deps = dependencies[key]
KeyError: 'ndarray-298394c3b8e42a8df373f5caf974b390'
distributed.scheduler - ERROR - 'Series-a7319e9ce901d40c64e898401542456b'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py

KeyError: 'ndarray-a572894a700a3684a98a51e1332c90f6'
distributed.scheduler - ERROR - 'ndarray-a572894a700a3684a98a51e1332c90f6'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    deps = dependencies[key]
KeyError: 'ndarray-a572894a700a3684a98a51e1332c90f6'
distributed.scheduler - ERROR - 'ndarray-a572894a700a3684a98a51e1332c90f6'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    d

KeyError: 'ndarray-a44dae35a221940f5c3d26c529744c57'
distributed.scheduler - ERROR - 'ndarray-a44dae35a221940f5c3d26c529744c57'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    deps = dependencies[key]
KeyError: 'ndarray-a44dae35a221940f5c3d26c529744c57'
distributed.scheduler - ERROR - 'ndarray-a44dae35a221940f5c3d26c529744c57'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    d

KeyError: 'ndarray-a44dae35a221940f5c3d26c529744c57'
distributed.scheduler - ERROR - 'ndarray-a44dae35a221940f5c3d26c529744c57'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    deps = dependencies[key]
KeyError: 'ndarray-a44dae35a221940f5c3d26c529744c57'
distributed.scheduler - ERROR - 'ndarray-a44dae35a221940f5c3d26c529744c57'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    d

...

KeyError: 'ndarray-a572894a700a3684a98a51e1332c90f6'
distributed.scheduler - ERROR - 'ndarray-bb8c0d6a250f613f642e4ff4bec74c91'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    deps = dependencies[key]
KeyError: 'ndarray-bb8c0d6a250f613f642e4ff4bec74c91'
distributed.scheduler - ERROR - 'ndarray-a572894a700a3684a98a51e1332c90f6'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    d

KeyError: 'ndarray-a44dae35a221940f5c3d26c529744c57'
distributed.scheduler - ERROR - 'ndarray-a44dae35a221940f5c3d26c529744c57'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    deps = dependencies[key]
KeyError: 'ndarray-a44dae35a221940f5c3d26c529744c57'
distributed.scheduler - ERROR - 'ndarray-a44dae35a221940f5c3d26c529744c57'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    d

KeyError: 'ndarray-a44dae35a221940f5c3d26c529744c57'
distributed.scheduler - ERROR - 'ndarray-a44dae35a221940f5c3d26c529744c57'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    deps = dependencies[key]
KeyError: 'ndarray-a44dae35a221940f5c3d26c529744c57'
distributed.scheduler - ERROR - 'ndarray-a44dae35a221940f5c3d26c529744c57'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    d

KeyError: 'ndarray-bb8c0d6a250f613f642e4ff4bec74c91'
distributed.scheduler - ERROR - 'ndarray-bb8c0d6a250f613f642e4ff4bec74c91'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    deps = dependencies[key]
KeyError: 'ndarray-bb8c0d6a250f613f642e4ff4bec74c91'
distributed.scheduler - ERROR - 'ndarray-bb8c0d6a250f613f642e4ff4bec74c91'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    d

KeyError: 'ndarray-b3d92a91c2a1497bbb9da921771d0610'
distributed.scheduler - ERROR - 'ndarray-b3d92a91c2a1497bbb9da921771d0610'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    deps = dependencies[key]
KeyError: 'ndarray-b3d92a91c2a1497bbb9da921771d0610'
distributed.scheduler - ERROR - 'ndarray-b3d92a91c2a1497bbb9da921771d0610'
Traceback (most recent call last):
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\scheduler.py", line 1544, in update_graph
    loose_restrictions=loose_restrictions)
  File "C:\Users\Lindstaedt\Anaconda3\envs\trader_env_intel\lib\site-packages\distributed\diagnostics\graph_layout.py", line 47, in update_graph
    d

In [22]:
print(cv.best_params_)
pickle.dump(cv.best_params_, open('../models/GridSearch_vbetaone.pkl', 'wb'))

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [34]:
print_feature_importances(cv.best_estimator_.named_steps['randomForest'])

Feature ranking:
1 importance: 0.019819236920683308. feature name: week
2 importance: 0.013932401202644324. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-16
3 importance: 0.012933832650140514. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-4
4 importance: 0.012711420987270555. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-6
5 importance: 0.012474221302207652. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-17
6 importance: 0.012403112877513944. feature name: AdjCloseSMA50_-19
7 importance: 0.012301925392493867. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-14
8 importance: 0.011554757394487972. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-16
9 importance: 0.010534778033378853. feature name: AdjCloseSMA50_-16
10 importance: 0.010152821412917862. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-5
11 importance: 0.009186356321784496. feature name: AdjCloseSMA200_to_AdjClose_ratio_-8
12 importance: 0.00884593884802256. feature name: AdjCloseSMA50_to

216 importance: 0.001697076331188201. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-7
217 importance: 0.0016928093648064927. feature name: AdjOpen_-13
218 importance: 0.0016735610926028397. feature name: AdjLow_chg_-12
219 importance: 0.0016650402347949493. feature name: AdjCloseSMA50_to_AdjClose_ratio_-10
220 importance: 0.0016405695238459988. feature name: AdjOpen_chg_-4
221 importance: 0.0016202008903382734. feature name: AdjClose_-1
222 importance: 0.0016195672333592162. feature name: AdjCloseSMA10_to_AdjClose_ratio_-9
223 importance: 0.0016078791529155587. feature name: AdjCloseSMA200_-1
224 importance: 0.0016039475038763153. feature name: AdjVolume_chg_-14
225 importance: 0.0016003383204128481. feature name: AdjCloseSMA200_chg_-5
226 importance: 0.0015997046358071283. feature name: AdjCloseSMA200_-18
227 importance: 0.0015905631215865682. feature name: NFLX.US
228 importance: 0.0015830622318671835. feature name: AdjCloseSMA10_-8
229 importance: 0.0015800591205098083. featu

456 importance: 0.0. feature name: AdjHigh_-17
457 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-14
458 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-15
459 importance: 0.0. feature name: AdjOpen_-2
460 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-16
461 importance: 0.0. feature name: AdjOpen_0
462 importance: 0.0. feature name: AdjCloseSMA50_chg_-6
463 importance: 0.0. feature name: AdjHigh_-18
464 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-17
465 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-12
466 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-18
467 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-19
468 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_0
469 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-1
470 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-2
471 impor

In [35]:
y_pred_GridSearch = cv.predict(X_test)

# save for backtesting in separate notebook
pickle.dump(y_pred_GridSearch, open('../data/model_predictions/y_pred_GridSearch_vbetaone.pkl', 'wb'))

C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [36]:
print(classification_report(y_test, y_pred_GridSearch))

              precision    recall  f1-score   support

       False       0.64      0.86      0.73      1153
        True       0.19      0.06      0.09       603

   micro avg       0.59      0.59      0.59      1756
   macro avg       0.41      0.46      0.41      1756
weighted avg       0.48      0.59      0.51      1756



In [37]:
accuracy_score(y_test, y_pred_GridSearch)

0.5876993166287016

In [38]:
confusion_matrix(y_test, y_pred_GridSearch, labels=[False, True])

array([[995, 158],
       [566,  37]], dtype=int64)

## next: resume in backtesting notebook to understand and measure true performance of the model predictions